In [1]:
import torch
print(torch.version.cuda)      # None 이면 CPU 전용 버전
print(torch.cuda.is_available())  # False이면 GPU 미인식
print(torch.backends.cudnn.enabled)  # True 여야 GPU 가속 활용

12.6
True
True


In [2]:
import numpy as np
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import pandas as pd

c:\Users\dgh00\Desktop\ai-news-sentiment-analysis\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# 1) CSV 로드 및 레이블 매핑 -1,0,1 -> 0,1,2
df = pd.read_csv("full_news_labeled.csv")

# 2) Hugging Face Dataset 변환
dataset = Dataset.from_pandas(df[['news', 'label']])

# 3) 토크나이저 및 모델 로드
model_name = "DataWizardd/finbert-sentiment-ko"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=3,
    id2label={0:"부정",1:"중립",2:"긍정"},
    label2id={"부정":0, "중립":1, "긍정":2}
)

In [4]:
# 4) 토큰화 함수 정의
def tokenize_function(examples):
    return tokenizer(examples['news'], padding="max_length", truncation=True, max_length=256)

In [5]:
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# 5) 데이터셋 분할 (train 80%, valid 10%, test 10%)
split = tokenized_dataset.train_test_split(test_size=0.2, seed=42)
valid_test = split['test'].train_test_split(test_size=0.5, seed=42)
train_dataset = split['train']
valid_dataset = valid_test['train']
test_dataset = valid_test['test']

# 6) 평가 지표 - 정확도와 macro F1
import evaluate
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy_metric.compute(predictions=preds, references=labels)["accuracy"]
    f1 = f1_metric.compute(predictions=preds, references=labels, average="macro")["f1"]
    return {"accuracy": acc, "f1_macro": f1}

# 7) 학습 인자
training_args = TrainingArguments(
    output_dir="./finetuned-finbert-eth",
    eval_strategy="epoch",       # 반드시 지정
    save_strategy="epoch",
    eval_steps=500,
    save_steps=500,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    logging_dir="./logs",
    logging_steps=50,
    fp16=True,
)

# 8) Trainer 객체 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# 9) 학습 시작
trainer.train()

# 10) 평가
eval_results = trainer.evaluate(test_dataset)
print(eval_results)

# 11) 모델 저장
trainer.save_model("./finetuned-finbert-eth")
tokenizer.save_pretrained("./finetuned-finbert-eth")


Map: 100%|██████████| 600/600 [00:00<00:00, 2908.99 examples/s]
C:\Users\dgh00\AppData\Local\Temp\ipykernel_1660\3828845159.py:42: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro
1,No log,0.778821,0.633333,0.571718
2,1.003900,0.754746,0.650000,0.625358
3,1.003900,0.764347,0.650000,0.625358


{'eval_loss': 0.6782199740409851, 'eval_accuracy': 0.7, 'eval_f1_macro': 0.6576879910213244, 'eval_runtime': 5.6844, 'eval_samples_per_second': 10.555, 'eval_steps_per_second': 0.352, 'epoch': 3.0}


('./finetuned-finbert-eth\\tokenizer_config.json',
 './finetuned-finbert-eth\\special_tokens_map.json',
 './finetuned-finbert-eth\\vocab.txt',
 './finetuned-finbert-eth\\added_tokens.json',
 './finetuned-finbert-eth\\tokenizer.json')

In [9]:
import optuna

def hp_search(trial):
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 5e-5, log=True)
    batch_size = trial.suggest_categorical("batch_size", [8, 16, 32])
    num_train_epochs = trial.suggest_int("num_train_epochs", 3, 10)

    training_args = TrainingArguments(
        output_dir="./optuna_output",
        eval_strategy="epoch",
        save_strategy="epoch",
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size * 2,
        num_train_epochs=num_train_epochs,
        weight_decay=0.01,
        load_best_model_at_end=True,
        metric_for_best_model="f1_macro",
        logging_steps=50,
        fp16=True,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=valid_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    trainer.train()
    eval_result = trainer.evaluate()
    return eval_result["eval_f1_macro"]

study = optuna.create_study(direction="maximize")
study.optimize(hp_search, n_trials=20)

print(study.best_params)


[I 2025-09-12 17:07:02,897] A new study created in memory with name: no-name-9b8edbbe-8247-40b9-aabc-62415024adaf
C:\Users\dgh00\AppData\Local\Temp\ipykernel_1660\1237348288.py:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro
1,No log,0.947404,0.633333,0.600948
2,0.319700,0.988884,0.616667,0.585185
3,0.319700,1.043674,0.633333,0.604248
4,0.172200,1.228279,0.616667,0.574761
5,0.101700,1.326571,0.616667,0.573862
6,0.101700,1.398075,0.616667,0.573862
7,0.048900,1.422717,0.616667,0.573862


[I 2025-09-12 17:29:04,305] Trial 0 finished with value: 0.6042483660130719 and parameters: {'learning_rate': 1.6602702431840275e-05, 'batch_size': 16, 'num_train_epochs': 7}. Best is trial 0 with value: 0.6042483660130719.
C:\Users\dgh00\AppData\Local\Temp\ipykernel_1660\1237348288.py:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro
1,No log,1.117427,0.666667,0.642807
2,No log,1.182399,0.616667,0.582473
3,No log,1.310919,0.633333,0.602153
4,0.101500,1.403636,0.650000,0.614815
5,0.101500,1.466667,0.666667,0.633027
6,0.101500,1.504442,0.666667,0.634574
7,0.029700,1.548639,0.650000,0.614815
8,0.029700,1.572813,0.650000,0.613452


[I 2025-09-12 17:57:07,127] Trial 1 finished with value: 0.6428070175438596 and parameters: {'learning_rate': 1.2603231319761329e-05, 'batch_size': 32, 'num_train_epochs': 8}. Best is trial 1 with value: 0.6428070175438596.
C:\Users\dgh00\AppData\Local\Temp\ipykernel_1660\1237348288.py:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


...

Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro
1,0.000000,4.902848,0.700000,0.689332
2,0.000000,4.926665,0.700000,0.689332
3,0.000000,4.945442,0.700000,0.689332
4,0.000000,5.145463,0.700000,0.683112
5,0.000000,5.088241,0.683333,0.670370
6,0.000000,5.134587,0.666667,0.648254


[I 2025-09-12 23:50:45,429] Trial 18 finished with value: 0.6893320299771913 and parameters: {'learning_rate': 2.2027442562684383e-05, 'batch_size': 8, 'num_train_epochs': 6}. Best is trial 12 with value: 0.7237507906388361.
C:\Users\dgh00\AppData\Local\Temp\ipykernel_1660\1237348288.py:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro
1,0.000000,4.951653,0.700000,0.689332
2,0.000000,4.984963,0.700000,0.689332
3,0.000000,5.003397,0.700000,0.689332
4,0.000000,5.134942,0.683333,0.660778


[I 2025-09-13 00:06:18,164] Trial 19 finished with value: 0.6893320299771913 and parameters: {'learning_rate': 4.110255994094871e-05, 'batch_size': 8, 'num_train_epochs': 4}. Best is trial 12 with value: 0.7237507906388361.


{'learning_rate': 3.537817816941843e-05, 'batch_size': 8, 'num_train_epochs': 5}


In [10]:
study.best_params

{'learning_rate': 3.537817816941843e-05,
 'batch_size': 8,
 'num_train_epochs': 5}